In [40]:
import findspark
findspark.init()
from pyspark import SparkContext
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, BooleanType
from pyspark.sql.functions import unix_timestamp, year, month, dayofmonth, dayofyear

In [2]:
db_path = "I:/__Database__/Data Science/Crimes_-_2001_to_present.csv"

In [3]:
spark = SparkSession.builder \
        .master("local") \
        .appName("chicago") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()

In [67]:
crime_schema = StructType([
    StructField("ID", StringType(), True),
    StructField("Case_Number", StringType(), True),
    StructField("Date", StringType(), True),
    StructField("Block", StringType(), True),
    StructField("IUCR", StringType(), True),
    StructField("Primary_Type", StringType(), True),
    StructField("Description", StringType(), True),
    StructField("Location_Description", StringType(), True),
    StructField("Arrest", StringType(), True),
    StructField("Domestic", StringType(), True),
    StructField("Beat", StringType(), True),
    StructField("District", StringType(), True),
    StructField("Ward", StringType(), True),
    StructField("Community_Area", StringType(), True),
    StructField("FBI_Code", StringType(), True),
    StructField("X_Coordinate", StringType(), True),
    StructField("Y_Coordinate", StringType(), True),
    StructField("Year", StringType(), True),
    StructField("Updated_On", StringType(), True),
    StructField("Latitude", StringType(), True),
    StructField("Longitude", StringType(), True),
    StructField("Location", StringType(), True),
])

crime_df = spark.read.csv(db_path, header=True, schema=crime_schema)

In [69]:
from_pattern = 'MM/dd/yyyy hh:mm:ss aa'
crime_df = crime_df.withColumn('DateTS', unix_timestamp(crime_df['Date'], from_pattern).cast("timestamp")).drop('Date')

In [70]:
crime_df.first()

Row(ID=u'2893074', Case_Number=u'HJ564203', Block=u'005XX W 36TH ST', IUCR=u'0610', Primary_Type=u'BURGLARY', Description=u'FORCIBLE ENTRY', Location_Description=u'RESIDENCE-GARAGE', Arrest=u'false', Domestic=u'false', Beat=u'0925', District=u'009', Ward=u'11', Community_Area=u'60', FBI_Code=u'05', X_Coordinate=u'1173107', Y_Coordinate=u'1881040', Year=u'2003', Updated_On=u'04/15/2016 08:55:02 AM', Latitude=u'41.829001329', Longitude=u'-87.640381824', Location=u'(41.829001329, -87.640381824)', DateTS=datetime.datetime(2003, 8, 15, 15, 0))

In [71]:
crime_df.groupBy(year(crime_df.DateTS)).count().orderBy("year(DateTS)").show()

+------------+------+
|year(DateTS)| count|
+------------+------+
|        2001|485726|
|        2002|486739|
|        2003|475912|
|        2004|469354|
|        2005|453676|
|        2006|448047|
|        2007|436942|
|        2008|426974|
|        2009|392571|
|        2010|370179|
|        2011|351593|
|        2012|335731|
|        2013|306750|
|        2014|274658|
|        2015|263208|
|        2016|267328|
|        2017| 85371|
+------------+------+



In [45]:
crime_df.groupBy(month(crime_df.DateTS)).count().orderBy("month(DateTS)").show()

+-------------+------+
|month(DateTS)| count|
+-------------+------+
|            1|506713|
|            2|445989|
|            3|535471|
|            4|536891|
|            5|559609|
|            6|551977|
|            7|580414|
|            8|574368|
|            9|539600|
|           10|547105|
|           11|491285|
|           12|461337|
+-------------+------+



In [57]:
crime_df.groupBy("Ward").count().orderBy("count").show(1000)

+----+-----+
|Ward|count|
+----+-----+
|0310|    5|
|0430|   19|
|1655|  767|
|1652|  977|
|1653| 2117|
|0235| 2387|
|1654| 2862|
|0215| 3207|
|1235| 3287|
|1915| 3500|
|1921| 3668|
|1225| 3881|
|1215| 3983|
|1221| 4090|
|0225| 4091|
|1214| 4139|
|0121| 4139|
|1234| 4150|
|0114| 4520|
|1934| 4590|
|1935| 4763|
|1925| 5643|
|1914| 5861|
|1621| 6631|
|2133| 7139|
|2323| 8240|
|2111| 8345|
|2322| 8500|
|1333| 8903|
|1612| 8983|
|2131| 9455|
|2022| 9676|
|1613| 9711|
|2313| 9828|
|1232|10079|
|2312|10099|
|2032|10109|
|2324|10190|
|1322|10297|
|2033|10298|
|2332|10354|
|2012|10879|
|1813|10931|
|1611|10938|
|0231|11067|
|2112|11190|
|2031|11210|
|2333|11240|
|2023|11784|
|2331|11844|
|1911|12237|
|0434|12360|
|1614|12634|
|2513|12694|
|0133|12700|
|2013|12919|
|2124|13173|
|1922|13292|
|2122|13467|
|1332|13508|
|1312|13531|
|0921|13532|
|0925|13757|
|1313|13854|
|1631|13864|
|0214|13868|
|2024|14039|
|1912|14041|
|1324|14093|
|1032|14103|
|1125|14384|
|0224|14571|
|1814|14640|
|2431|14867|

In [47]:
crime_df.groupBy("Beat").count().orderBy("Beat").show()

+-----+-------+
| Beat|  count|
+-----+-------+
|false|5512611|
| true| 818148|
+-----+-------+



In [55]:
crime_df.groupBy("Primary_Type").count().orderBy("count").show(1000, truncate=False)

+---------------------------------+-------+
|Primary_Type                     |count  |
+---------------------------------+-------+
|DOMESTIC VIOLENCE                |1      |
|NON-CRIMINAL (SUBJECT SPECIFIED) |7      |
|RITUALISM                        |23     |
|HUMAN TRAFFICKING                |32     |
|NON - CRIMINAL                   |38     |
|NON-CRIMINAL                     |99     |
|CONCEALED CARRY LICENSE VIOLATION|105    |
|OTHER NARCOTIC VIOLATION         |113    |
|PUBLIC INDECENCY                 |144    |
|OBSCENITY                        |438    |
|STALKING                         |3084   |
|INTIMIDATION                     |3687   |
|KIDNAPPING                       |6388   |
|HOMICIDE                         |8435   |
|ARSON                            |10544  |
|INTERFERENCE WITH PUBLIC OFFICER |13266  |
|LIQUOR LAW VIOLATION             |13691  |
|GAMBLING                         |14073  |
|SEX OFFENSE                      |23229  |
|CRIM SEXUAL ASSAULT            

In [59]:
crime_df.groupBy("Location_Description").count().orderBy("count").show(1000, False)

+-----------------------------------------------+-------+
|Location_Description                           |count  |
+-----------------------------------------------+-------+
|TRUCKING TERMINAL                              |1      |
|CLEANERS/LAUNDROMAT                            |1      |
|EXPRESSWAY EMBANKMENT                          |1      |
|POOLROOM                                       |1      |
|ELEVATOR                                       |1      |
|JUNK YARD/GARBAGE DUMP                         |1      |
|PUBLIC GRAMMAR SCHOOL                          |1      |
|LAGOON                                         |1      |
|FUNERAL PARLOR                                 |1      |
|AUTO / BOAT / RV DEALERSHIP                    |1      |
|LIVERY AUTO                                    |1      |
|LOADING DOCK                                   |1      |
|BANQUET HALL                                   |1      |
|GOVERNMENT BUILDING                            |1      |
|ROOMING HOUSE